In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import json

comments = []
posts = []
for f in ['najib-razak.json', 'anwar-ibrahim.json', 'sanusi.json', 'pakatan-harapan.json',
         'rafizi-ramli.json', 'mymgag.json', 'Ayahanda-Abdul-Rani-Kulup-1753773051575006.json',
         'groups-1483603055330698.json', 'groups-392252305221999.json', 'groups-1098480360644592.json',
         'groups-2025865557592801.json']:
    with open(f) as fopen:
        data = json.load(fopen)
    
    for r in data:
        if 'groups-' not in f:
            posts.append(r['post_text'])
        for c in r['comments_full']:
            t = c['comment_text']
            if 1 < len(t.split()) < 300:
                comments.append(t)
            for c_ in c['replies']:
                t = c_['comment_text']
                if 1 < len(t.split()) < 300:
                    comments.append(t)

In [3]:
len(posts), len(comments)

(1255, 263917)

In [4]:
comments = list(set(comments))
len(comments)

199590

In [5]:
import malaya
from malaya.text.function import maxlen_slide

In [6]:
import re

def fix_spacing_punct(string):
    string = string.replace(' , ', ', ').replace(' !', '!').replace(' . ', '. ').replace('( ', '(') \
    .replace(' )', ')')
    return re.sub(r'[ ]+', ' ', string).strip()

In [7]:
comments[0]

'Mohd Aquil Mohd Yusof sabar cho..pru 16 kita sapu bersih..'

In [8]:
normalizer = malaya.normalize.normalizer(date = False)

In [9]:
segmenter = malaya.segmentation.huggingface()
segmenter_func = lambda x: segmenter.generate([x], max_length = 256)[0]

In [10]:
from unidecode import unidecode

unidecode('Denga kata YB rebah bing marin, semoga YB xdak apa² yg jadi laa,')

'Denga kata YB rebah bing marin, semoga YB xdak apa2 yg jadi laa,'

In [11]:
t = 'Tahniah Pm.10'
t = unidecode(t)

In [12]:
normalizer._tokenizer(t)

['Tahniah', 'Pm', '.', '10']

In [13]:
normalizer.normalize(
            t, normalize_hingga = False, normalize_cardinal = False,
            normalize_ordinal = False, normalize_pada_hari_bulan = False,
            normalize_fraction = False, normalize_money = False, normalize_date = False,
            normalize_time = False, normalize_ic = False,
            normalize_units = False,
            normalize_url = False, normalize_percent=False, normalize_telephone = False,
            segmenter = segmenter_func,
            not_a_word_threshold = 1e-9,
        )['normalize']

'Tahniah Pm . 10'

In [15]:
from tqdm import tqdm

results = []
already_in = set()
for t in tqdm(comments):

    if t.lower() in already_in:
        continue

    if not len(t):
        continue
        
    t = unidecode(t)

    try:
        normalized = normalizer.normalize(
            t, normalize_hingga = False, normalize_cardinal = False,
            normalize_ordinal = False, normalize_pada_hari_bulan = False,
            normalize_fraction = False, normalize_money = False, normalize_date = False,
            normalize_time = False, normalize_ic = False,
            normalize_units = False,
            normalize_url = False, normalize_percent=False, normalize_telephone = False,
            normalize_elongated = True,
            not_a_word_threshold = 1e-9,
        )['normalize']

        normalized = fix_spacing_punct(normalized)

        already_in.add(t.lower())
    except Exception as e:
        print(e)
        print(t)
        
        normalized = t
    
    results.append({'original': t, 'text': normalized})

100%|███████████████████████████████████████████████████████████████████████████████| 199590/199590 [02:09<00:00, 1535.37it/s]


In [16]:
len(results)

195971

In [17]:
results[-100:]

[{'original': 'Fully support PH\nRakyat hidup\nKita boleh ',
  'text': 'Fully support PAKATAN HARAPAN Rakyat hidup Kita boleh'},
 {'original': 'Sprm pun bagi masa utk berkempen..tapi\nawai dah kayuh.....',
  'text': 'Sprm pun bagi masa untuk berkempen. tetapi awai dah kayuh. .. . .'},
 {'original': 'Kena pujuk vs kena heret ', 'text': 'Kena pujuk vs kena heret'},
 {'original': 'Bos Getah aamiin aamiin', 'text': 'Bos Getah aamiin aamiin'},
 {'original': 'aku berani jamin dengan live pru15 bawah 50K ni, ramai pengundi dah berubah',
  'text': 'aku berani jamin dengan live pru15 bawah 50K ini, ramai pengundi dah berubah'},
 {'original': 'Bertahan guys! Walau beratur panjang ia berbaloi, klu X nnt menyesal 5 tahun',
  'text': 'Bertahan guys! Walau beratur panjang ia berbaloi, kalau Tak nanti menyesal 5 tahun'},
 {'original': 'Lu tanya sama lu punya atok..sejarah ini batu pahat',
  'text': 'Kamu tanya sama kamu punya atok. . sejarah ini batu pahat'},
 {'original': 'Adus Affandy mcm ktinggala

In [18]:
with open('facebook-data-v2.jsonl', 'w') as fopen:
    for r in results:
        fopen.write(f'{json.dumps(r)}\n')

In [19]:
!split -l 50000 -d --additional-suffix=.splitted facebook-data-v2.jsonl facebook-data-v2.jsonl